In [ ]:
import pandas as pd
import pywhatkit as wpp
from datetime import datetime 

#SetOptions para ver los dataframes
#pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth',False)

Funciones

In [ ]:
def clean_extras(df_e):
    # Columnas importantes
    df_e = df_e[["Matricula","Nombre","Calificacion", "Materia","Clave Carrera", "Tipo Calificacion", "Centro Enlace"]]
    
    # Filtro CE
    df_e = df_e[df_e["Centro Enlace"].isin(["CADEREYTA", "CC CADEREYTA"])]
    
    # Agrupacion de Materias
    df_agrupado = df_e.groupby("Nombre")["Materia"].agg(lambda x: ", ".join(x)).reset_index().rename(columns = {"Materia": "Materias"})
    df_e = pd.merge(df_e, df_agrupado, on = "Nombre", how = "left")     

    #Cambio de columna y eliminar la extra
    columna_extra = df_e["Materias"]
    df_e.drop(columns = ["Materias"], inplace=True)
    df_e["Materia"] = columna_extra

    #Filtra dúplicados
    df_e = df_e.drop_duplicates( subset = "Nombre")
    
    #Renombra
    df_e.rename(columns = {"Materia" : "Materias"}, inplace= True)
    return df_e

In [ ]:
def clean_database(db):
    #Renombrar el nombre de la columna NOMBRE
    db.rename(columns = {'NOMBRE                                                                                                                                      ' : "NOMBRE", 'MATRICULA' : "Matricula"}, inplace= True)

    # Columnas importantes
    db = db[['NOMBRE', 'Matricula', 'CE CANALIZADO', 'CARRERA', f'M{mes}', 'Correo', 'Teléfono', 'Teléfono2', 'Teléfono3', 'CURP', 'Agente ', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar']]

    # Filtro Cadereyta
    # db = db[db['CE CANALIZADO'] == 'CE CADEREYTA']
    cade = ('CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA')
    db = db[db['CE CANALIZADO'].str.contains('|'.join(cade), case = False, na = False)]
    return db

In [ ]:
def save_extras_cont(df):
    df2.append(datetime.now().strftime('%d/%m/%Y'))
    df2 = pd.DataFrame(df, columns=['Matricula'])
    df2.to_csv(path2 + 'extras_contactados.csv', index=False)

In [ ]:
def search(df, matricula):
    return df[df['Matricula'] == matricula]

Lectura del Excel

In [ ]:
path = 'Data/'
main = 'Main/'
try:
    database = pd.read_csv(path + "BASE CNCI.csv") 
except FileNotFoundError or FileExistsError: 
    %run DatabaseCNCI.ipynb
    print('Database CNCI.csv Created')
    database = pd.read_csv(path + "BASE CNCI.csv")

try: 
    extras = pd.read_csv(path + "Extras.csv")
except FileNotFoundError or FileExistsError:
    mes = datetime.now().month
    extras = pd.read_excel(path + main + "Román - M{} Seguimiento.xlsx".format(mes), sheet_name = "Extras")
    extras = clean_extras(extras)
    extras.to_csv(path + 'Extras.csv', index=False)
    print('Extras csv Created')

In [ ]:
mes = datetime.now().month
database = clean_database(database)

Reprobados DataFrame

In [ ]:
# Array de reprobados
matricula_reprobados = extras['Matricula'].unique()

# Filtro a los reprobados y M4
reprobados_mod4 = database[(database['Matricula'].isin(matricula_reprobados)) & (database[f'M{mes}'] == '4')]

# Mergear los DF
reprobados_mod4 = pd.merge(extras, reprobados_mod4, on = 'Matricula')

# Columnas necesarias
reprobados_mod4 = reprobados_mod4[['Matricula', 'Nombre', 'Calificacion', 'Materias', 'Clave Carrera', 'Tipo Calificacion', 'Centro Enlace', f'M{mes}', 'Correo', 'Teléfono', 'Teléfono2', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar']]

Archivos de utilidad

In [ ]:
path2 = 'infodata/'
imagen = 'img/Extras.jpeg'

matric_error = pd.read_csv(path2 + 'matriculas_error.csv')
matric_error = matric_error.dropna().values.flatten().tolist()
try:
    matric_yacontactadas = pd.read_csv(path2 + 'extras_contactados.csv')
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist()
except FileNotFoundError or FileExistsError:
    matric_yacontactadas = list()

Mensaje predeterminado para alumnos

In [ ]:
for index, row in reprobados_mod4.iterrows():
    alumno = row['Nombre']
    matricula = row['Matricula']
    tipoExamen = row['Tipo Calificacion']
    materias = row['Materias']
    af = row['Asesor Financiero ']
    telefono = row['Teléfono']
    
    if not pd.isna(telefono):
        telefono = '+52 '+str(telefono)
    if matricula in (matric_error):
        if not pd.isna(row['Teléfono2']):
            telefono2 ='+52 '+str(row['Teléfono2'])
            # print(telefono, telefono2)
            telefono = telefono2
            print('Matricula con error: {}, Alumno: {}'.format(matricula, alumno))

    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, alumno: {}'.format(matricula, alumno))
    else:
        mensaje = 'Se acerca la fecha de *Extraordinarios*. ¿Cómo te encuentras ' + alumno + ' ' + matricula + '? \nTe habla _Román Rodríguez_ Tutor Presencial del CNCI, me comunico contigo debido a que veo que dejaste en ' + tipoExamen + ' la(s) siguiente(s) materia(s): *' + materias +'*. \n¿Me gustaría saber si puedo apoyarte con el proceso de regularización? \nTe dejo las fechas siguientes:'
        print('Enviado a: {}, {}'.format(matricula, alumno))
        matric_yacontactadas.append(matricula)
        # wpp.sendwhats_image(telefono, imagen, mensaje, 25, True)

save_extras_cont()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Lee la imagen
# print(alumno, resultado)
figure = Image.open(imagen)
# Muestra la imagen
plt.imshow(figure)
plt.axis('off')  # Desactiva los ejes
plt.show()

# Consultas

In [ ]:
reprobados_mod4[reprobados_mod4['Matricula']== 'BNL093535'][['Matricula','Nombre', 'Asesor Financiero ']]

In [ ]:
extras[extras['Matricula']== 'BNL088083']

In [ ]:
database[database['Matricula']== 'BNL088083']

Avances nuevos

In [ ]:
# Outlook Email : https://recursospython.com/guias-y-manuales/enviar-correo-electronico-via-outlook-y-smtp/

def extra_AF(destinatario):
    from email.message import EmailMessage
    import smtplib

    remitente = str(input("Correo?: ")) 
    password = str(input("Contraseña?: "))

    mensaje = "¡Hola, mundo!"
    email = EmailMessage()
    email["From"] = remitente
    email["To"] = destinatario
    email["Subject"] = "Correo de prueba"
    email.set_content(mensaje)
    # smtp = smtplib.SMTP("smtp-mail.outlook.com", port=587)
    smtp = smtplib.SMTP("smtp.office365.com", port=587)
    smtp.starttls()
    smtp.login(remitente, password)
    smtp.sendmail(remitente, destinatario, email.as_string())

In [ ]:
# Prototipo de funcion para mostrar todo el contenido
def extend(df):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    # pd.set_option('display.width', None)
    # pd.set_option('display.max_colwidth',False)
    df2 = df
    return df2